# Majorana Operator 

In [1]:
from symmer.symplectic import MajoranaOp
import numpy as np

# notes:

https://arxiv.org/pdf/2101.09349.pdf (pg11)

https://arxiv.org/abs/2110.10280

The Majorana operators {γ0, γ1, . . . , γm−1}, for m even, are linear Hermitian operators acting on the
fermionic Fock space 

$$H_{m/2} = \{ |b\rangle : b ∈ F_{2}^{m/2} \}$$

or equivalently the $m/2$-qubit complex Hilbert space satisfying $\forall 0 \leq i < j \leq m-1$:

1. $\gamma_{i}^{2} = \mathcal{I}$ - self inverse!
2. $\gamma_{i}\gamma_{j} = -\gamma_{j}\gamma_{i}$ - anti-commute!


These M single-mode operators generate a basis (up to phase factors) for the full algebra of Majorana operators via arbitrary products (https://arxiv.org/pdf/1908.08067.pdf pg9), (https://arxiv.org/pdf/1908.08067.pdf) and more importantly (https://arxiv.org/pdf/2102.00620.pdf):

$$\gamma_{A} = i^{\lfloor\frac{|\gamma_{A}|}{2}\rfloor}\prod_{k \in A}^{M-1} \gamma_{k}$$

where $A \subseteq \{0,1,...,M-1 \} $ and represents the "support" of $\gamma_{A}$. We write this as $|A|$ where is the hamming weight of $\gamma_{A}$.

Note: phase factor not included in https://arxiv.org/pdf/1908.08067.pdf, but is in https://arxiv.org/pdf/2102.00620.pdf!

`MajoranaOp` class stores Majorana operator as a symplectic array and vector of coefficients.

rows of symplectic array give an individual operator and associated coefficient in coeff vec gives coefficient (that will **by default have phase factor included in it**!)

therefore symplectic matrix is size $N \times M$ for $N$ terms and $M$ majorana sites (for $M/2$ fermionic sites)

In [2]:
# Class defaults to calculate phase factors when initalized, as so:

y1_y2_y3_y4 = [[0,3,5]]
coeff = [1]
Maj_with_phase = MajoranaOp.from_list(y1_y2_y3_y4, coeff, phase_factors_included=False) 
print(Maj_with_phase)


1j γ0 γ3 γ5


In [3]:
# can override so that phase has already been included
Maj_withOUT_phase = MajoranaOp.from_list(y1_y2_y3_y4, coeff, phase_factors_included=True) 
print(Maj_withOUT_phase)

# note internal functions of class use this method when multiplying and adding Majorana operators!

(1+0j) γ0 γ3 γ5


# NOTE order matters!

changing order of majorana operators generates a sign:

$$\gamma_{i}\gamma_{j} = -\gamma_{j}\gamma_{i}$$

`MajoranaOp` orders the majorana operators by increase index (normal order) therefore if not in normal order form a re-ordering is done, where sign is kept track of

- code uses bubble sort to count number of times order change happens (generating a -1 sign each time)


In [4]:
ops = [
            [4,3]# op1
           ]

Maj = MajoranaOp.from_list([ops[0][::-1]], [1])
print(Maj)

Maj_flipped = MajoranaOp.from_list(ops, [1])
print(Maj_flipped)
# order flipped! generating a sign!

1j γ3 γ4
(-0-1j) γ3 γ4


In [5]:
Maj

1j γ3 γ4

# Hermitian and unitarity

In [88]:
operators = [
#             [1,2,3,4,5,7], # op1
            [1,2,3,4,12]
           ]
coeffs = [1j]
Maj = MajoranaOp.from_list(operators, coeffs, phase_factors_included=True)
print(Maj)

# unitary!
print('self inverse (up to sign) check:', Maj*Maj)

# unitary check!
print('unitary check:', Maj * Maj.conjugate)

# note if majorana op has complex phase in coefficient, 
# then hermitian check will return False, as phase sign is flipped

1j γ1 γ2 γ3 γ4 γ12
self inverse (up to sign) check: (-1+0j) I
unitary check: (1+0j) I


In [82]:
from openfermion import MajoranaOperator, hermitian_conjugated, QubitOperator, jordan_wigner
terms = [1,2,3,4,5]
Q = MajoranaOperator(term=tuple(terms)) *1
m1 = jordan_wigner(Q)
m2 = hermitian_conjugated(m1)

D = MajoranaOperator(term=tuple(terms[::-1])) *1
jordan_wigner(D) == m2

True

In [50]:
from openfermion import MajoranaOperator, hermitian_conjugated

test = MajoranaOperator(term=(1,2,3)) *1
test * test

MajoranaOperator.from_dict(terms={(): -1.0})

In [15]:
new_terms = []
for sym_vec in Maj.symp_matrix:
    current_term = Maj.term_index_list[sym_vec]
    new_terms.append(current_term[::-1]) # reverse order
# note when reversed performs bubble sort to put back into correct order
Maj_conj = MajoranaOp.from_list(new_terms, 
                                Maj.coeff_vec.conjugate(), 
                                phase_factors_included=True)
Maj_conj

(-1+0j) γ1 γ2 γ3

In [ ]:
operators = [
#             [1,2,3,4,5,7], # op1
            [3,2,1]
           ]
coeffs = [-1]
Maj = MajoranaOp.from_list(operators, coeffs, phase_factors_included=True)
Maj

# commutation

The anticommutator between two arbitrary Majorana operators $\gamma_{A}$ and $\gamma_{B}$ is determined by their individual supports and their overlap:



$$\{ \gamma_{A}, \gamma_{B} \} = \Big(1 + (-1)^{|A|\dot|B|- |A\cap B|} \Big) \gamma_{A}\gamma_{B}$$

Therefore:
- if $|A|\dot|B|- |A\cap B| = 0$
    - then terms anticommute
    
-else $|A|\dot|B|- |A\cap B| = 1$
    - and terms commute


# check commutation relations

uses above definition!

### 1. termwise commutation!

In [ ]:
operators = [
            [1,2,3,4], # op1
            [5,6,10] # op2
           ]
coeffs = [1,1]
Maj1 = MajoranaOp(operators, coeffs)


operators2 = [
            [0], # op1
            [2], # op2
            [3], # op3
           ]
coeffs2 = [1,
           2,
           3+1j]
Maj2 = MajoranaOp(operators2, coeffs2)

In [ ]:
print(Maj1)
print()
print(Maj2)

In [ ]:
Maj1.commutes_termwise(Maj2)

In [ ]:
print(Maj1.commutator(Maj2))

In [ ]:
operator = [
    [0,1,2,3],
    [5,6],
    [3,5],
    [0,1,2,3,4,5,6,7],
    [2,3],
    [4]
]

coeffs = np.arange(2,len(operator)+2)

###
M = MajoranaOp(operator, coeffs)
print(M)

M.adjacency_matrix()

In [ ]:
print(M*M)
print()

C = M.to_OF_op()
out = C*C
print(out)

In [ ]:
M.adjacency_matrix()

# convert from Fermions to Majoranas

In [ ]:
from openfermion import FermionOperator, get_majorana_operator
from symmer.symplectic import convert_openF_fermionic_op_to_maj_op

ham = (FermionOperator('0^ 3', .5) +
       FermionOperator('3^ 0', 0.5) +
      FermionOperator('3^ 2^ 0 1', 0.5))

M_out = convert_openF_fermionic_op_to_maj_op(ham, phase_factors_included=True)

# reason this is False is due to MajoranaOp including phase factors!
print(M_out.to_OF_op() == get_majorana_operator(ham))

# get basis for operator

In [ ]:
from symmer.utils import gf2_basis_for_gf2_rref, gf2_gaus_elim
import os
import json
from openfermion import reverse_jordan_wigner, jordan_wigner
from symmer.symplectic import PauliwordOp
from functools import reduce

In [ ]:
working_dir = os.getcwd()
parent_dir = os.path.dirname(working_dir)
data_dir = os.path.join(parent_dir,'data')

file = 'O1_STO-3G_triplet_OO.json'
# file = 'H2-Be1_STO-3G_singlet_BeH2BeH2.json'

file_path = os.path.join(data_dir, file)
with open(file_path, 'r') as input_file:
    ham_dict = json.load(input_file)
    
ham = PauliwordOp(ham_dict)

ham_opemF = ham.PauliwordOp_to_OF

H_qubit = reduce(lambda x,y: x+y, ham_opemF)

fermionic_H = reverse_jordan_wigner(H_qubit)
maj_H = convert_openF_fermionic_op_to_maj_op(fermionic_H)

In [ ]:
ZX_symp = maj_H.symp_matrix
reduced = gf2_gaus_elim(ZX_symp)
kernel  =  gf2_basis_for_gf2_rref(reduced)

kernel = kernel.astype(int)
kernel

In [ ]:
if kernel.shape[0]:
    basis_coeffs = np.ones(kernel.shape[0])
else:
    basis_coeffs=[1]

basis_op = MajoranaOp(kernel, basis_coeffs)
print(basis_op)

In [ ]:
openF_M_op = maj_H.to_OF_op()
basis_op_openF =  basis_op.to_OF_op()
print('commmutes: ', openF_M_op*basis_op_openF == basis_op_openF*openF_M_op)

# print(maj_H.commutes_termwise(basis_op))
maj_H.commutes(basis_op)

In [ ]:
jordan_wigner(basis_op.to_OF_op())

## Get cliffords to map to terms!

Section C of https://arxiv.org/pdf/2110.10280.pdf

A Majorana fermion stabilizer code [32], or Majorana stabilizer code for brevity, is the simultaneous +1 eigenspace of a collection of commuting, Hermitian, even **weight Majorana operators**. The evenness constraint ensures that these operators are fermion-parity preserving, and hence physically observable. 

Therefore rotate down onto pairs of majorana modes!

In [ ]:
from symred.majorana_operator import majorana_rotations

In [ ]:
basis_rot = majorana_rotations(basis_op)
final_terms, rotations = basis_rot.get_rotations()


rotated_basis = basis_op.copy()
for rot in rotations:
    rotated_basis = rot *rotated_basis * rot.conjugate
    
print(jordan_wigner(rotated_basis.to_OF_op()))

In [ ]:
## random test!

from openfermion import reverse_jordan_wigner, jordan_wigner, QubitOperator

basis = QubitOperator('X0 X1',1 ) + QubitOperator('Z0 Z1',1 ) + QubitOperator('Y3',1 )
basis_ferm = reverse_jordan_wigner(basis)
maj_test = convert_openF_fermionic_op_to_maj_op(basis_ferm, phase_factors_included=True)


# maj_test = MajoranaOp([[1,2,3,4],[3,4,5,6], [2,3,5,7]], [1,1,1])
# print(jordan_wigner(maj_test.to_OF_op()))

test_rot = majorana_rotations(maj_test)
final_terms, rotations = test_rot.get_rotations()


rotated_op = maj_test.copy()
for rot in rotations:
    rotated_op = rot *rotated_op * rot.conjugate
    
print(jordan_wigner(rotated_op.to_OF_op()))

In [ ]:
symp_matrix = np.arange(9).reshape(3,3)
N = np.einsum('ij->i', Maj.symp_matrix.astype(int))

N%2

In [ ]:
symp_matrix

In [ ]:
Maj

In [ ]:
Maj.conjugate

In [ ]:
operators = [
            [1,2,3,4,5,7], # op1
           ]
coeffs = [1j]
Maj = MajoranaOp.from_list(operators, coeffs)
print(Maj)
